# Airbnb Analysis

# IMPORTING MODULES

In [1]:
import pandas as pd
import numpy as np
import pymongo
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS, WordCloud
import warnings
warnings.filterwarnings('ignore')

# MongoDB connection

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['Airbnb_Analysis']
col = db['listingsAndReviews']

# Total Documents

In [3]:
Documents = [i for i in col.find()]
len(Documents)

102599

# Retrieve the Airbnb dataset for MongoDB connection

In [4]:
rel_data = []
for i in col.find():
    data = {
        'Id': i['_id'],
        'Name': i.get('NAME'),
        'Host_id': i.get('host id'),
        'Host_identity_verified': i.get('host_identity_verified'),
        'Host_name' : i.get('host name'),
        'Neighbourhood_group' : i.get('neighbourhood group'),
        'Neighbourhood' : i.get('neighbourhood'),
        'Latitube' : i.get('lat'),
        'Longtitube' : i.get('long'),
        'Country' : i.get('country'),
        'Country_code' : i.get('country code'),
        'Instant_bookable' : i.get('instant_bookable'),
        'Cancellation_policy' : i.get('cancellation_policy'),
        'Room_type' : i.get('room type'),
        'Construction_year' : i.get('Construction year'),
        'Price' : i.get('price'),
        'Service_fee' : i.get('service fee'),
        'Minimum_nights' : i.get('minimum nights'),
        'Number_of_reviews' : i.get('number of reviews'),
        'Last_review' : i.get('last review'),
        'Reviews_per_month' : i.get('reviews per month'),
        'Review_rate_number' : i.get('review rate number'),
        'Calculated_host_listings_count' : i.get('calculated host listings count'),
        'Availability_365' : i.get('availability 365'),
        'House_rules' : i.get('house_rules')
        
    }
    rel_data.append(data)


In [5]:
df = pd.DataFrame(rel_data)
df.head(5)

,Id,Name,Host_id,Host_identity_verified,Host_name,Neighbourhood_group,Neighbourhood,Latitube,Longtitube,Country,...,Price,Service_fee,Minimum_nights,Number_of_reviews,Last_review,Reviews_per_month,Review_rate_number,Calculated_host_listings_count,Availability_365,House_rules
0,6598f7d1fa398e3d8899aa10,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,6598f7d1fa398e3d8899aa11,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
2,6598f7d1fa398e3d8899aa12,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,None,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$620,$124,3.0,0.0,None,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and..."
3,6598f7d1fa398e3d8899aa13,None,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$368,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,None
4,6598f7d1fa398e3d8899aa14,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."


# Data Collection and Preprocessing

# Data type Correction

In [6]:
# checking Data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 25 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Id                              102599 non-null  object 
 1   Name                            102350 non-null  object 
 2   Host_id                         102599 non-null  int64  
 3   Host_identity_verified          102310 non-null  object 
 4   Host_name                       102193 non-null  object 
 5   Neighbourhood_group             102570 non-null  object 
 6   Neighbourhood                   102583 non-null  object 
 7   Latitube                        102591 non-null  float64
 8   Longtitube                      102591 non-null  float64
 9   Country                         102067 non-null  object 
 10  Country_code                    102468 non-null  object 
 11  Instant_bookable                102494 non-null  object 
 12  Cancellation_pol

In [7]:
# The below features are in Decimal128 type hence changing it to relevant data types
df['Name'] = df['Name'].isna().astype(str)
df['Host_identity_verified']  = df['Host_identity_verified'].isna().astype(str)
df['Host_name']  = df['Host_name'].isna().astype(str)
df['Neighbourhood_group'] = df['Neighbourhood_group'].isna().astype(str)
df['Neighbourhood']  = df['Neighbourhood'].isna().astype(str)
df['Latitube']  = df['Latitube'].isna().astype('Int64')
df['Longtitube']   = df['Longtitube'].isna().astype('Int64')
df['Country']  = df['Country'].isna().astype(str)
df['Country_code']  = df['Country_code'].isna().astype(str)
df['Instant_bookable']  = df['Instant_bookable'].isna().astype(str)
df['Cancellation_policy']  = df['Cancellation_policy'].isna().astype(str)
df['Construction_year']   = df['Construction_year'].isna().astype(float)
df['Price'] = df['Price'].fillna(0)
df['Service_fee']  = df['Service_fee'].isna().astype(str)
df['Minimum_nights']  = df['Minimum_nights'].isna().astype(float)
df['Number_of_reviews']  = df['Number_of_reviews'].isna().astype(float)
df['Last_review']  = df['Last_review'].isna().astype(str)
df['Reviews_per_month']  = df['Reviews_per_month'].isna().astype(float)
df['Review_rate_number']  = df['Review_rate_number'].isna().astype(float)
df['Calculated_host_listings_count']  = df['Calculated_host_listings_count'].isna().astype(float)
df['Availability_365']  = df['Availability_365'].isna().astype(float)
df['House_rules']  = df['House_rules'].isna().astype(float)

# Filling Missing values

In [8]:
df.isna().sum()

Id                                0
Name                              0
Host_id                           0
Host_identity_verified            0
Host_name                         0
Neighbourhood_group               0
Neighbourhood                     0
Latitube                          0
Longtitube                        0
Country                           0
Country_code                      0
Instant_bookable                  0
Cancellation_policy               0
Room_type                         0
Construction_year                 0
Price                             0
Service_fee                       0
Minimum_nights                    0
Number_of_reviews                 0
Last_review                       0
Reviews_per_month                 0
Review_rate_number                0
Calculated_host_listings_count    0
Availability_365                  0
House_rules                       0
dtype: int64

In [9]:
df.dtypes

Id                                 object
Name                               object
Host_id                             int64
Host_identity_verified             object
Host_name                          object
Neighbourhood_group                object
Neighbourhood                      object
Latitube                            Int64
Longtitube                          Int64
Country                            object
Country_code                       object
Instant_bookable                   object
Cancellation_policy                object
Room_type                          object
Construction_year                 float64
Price                              object
Service_fee                        object
Minimum_nights                    float64
Number_of_reviews                 float64
Last_review                        object
Reviews_per_month                 float64
Review_rate_number                float64
Calculated_host_listings_count    float64
Availability_365                  

In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Host_id,102599.0,49254111474.328667,28538996644.374817,123600518.0,24583328475.0,49117739352.0,73996495817.0,98763129024.0
Latitube,102599.0,0.000078,0.00883,0.0,0.0,0.0,0.0,1.0
Longtitube,102599.0,0.000078,0.00883,0.0,0.0,0.0,0.0,1.0
Construction_year,102599.0,0.002086,0.045623,0.0,0.0,0.0,0.0,1.0
Minimum_nights,102599.0,0.003986,0.063012,0.0,0.0,0.0,0.0,1.0
Number_of_reviews,102599.0,0.001784,0.042196,0.0,0.0,0.0,0.0,1.0
Reviews_per_month,102599.0,0.154768,0.361685,0.0,0.0,0.0,0.0,1.0
Review_rate_number,102599.0,0.003177,0.056279,0.0,0.0,0.0,0.0,1.0
Calculated_host_listings_count,102599.0,0.003109,0.055674,0.0,0.0,0.0,0.0,1.0
Availability_365,102599.0,0.004367,0.065936,0.0,0.0,0.0,0.0,1.0


In [11]:
rel_data[0]

{'Id': ObjectId('6598f7d1fa398e3d8899aa10'),
 'Name': 'Clean & quiet apt home by the park',
 'Host_id': 80014485718,
 'Host_identity_verified': 'unconfirmed',
 'Host_name': 'Madaline',
 'Neighbourhood_group': 'Brooklyn',
 'Neighbourhood': 'Kensington',
 'Latitube': 40.64749,
 'Longtitube': -73.97237,
 'Country': 'United States',
 'Country_code': 'US',
 'Instant_bookable': False,
 'Cancellation_policy': 'strict',
 'Room_type': 'Private room',
 'Construction_year': 2020,
 'Price': '$966 ',
 'Service_fee': '$193 ',
 'Minimum_nights': 10,
 'Number_of_reviews': 9,
 'Last_review': '10/19/2021',
 'Reviews_per_month': 0.21,
 'Review_rate_number': 4,
 'Calculated_host_listings_count': 6,
 'Availability_365': 286,
 'House_rules': "Clean up and treat the home the way you'd like your home to be treated.  No smoking."}

In [12]:
df.isnull().sum()

Id                                0
Name                              0
Host_id                           0
Host_identity_verified            0
Host_name                         0
Neighbourhood_group               0
Neighbourhood                     0
Latitube                          0
Longtitube                        0
Country                           0
Country_code                      0
Instant_bookable                  0
Cancellation_policy               0
Room_type                         0
Construction_year                 0
Price                             0
Service_fee                       0
Minimum_nights                    0
Number_of_reviews                 0
Last_review                       0
Reviews_per_month                 0
Review_rate_number                0
Calculated_host_listings_count    0
Availability_365                  0
House_rules                       0
dtype: int64

In [13]:
df.columns

Index(['Id', 'Name', 'Host_id', 'Host_identity_verified', 'Host_name',
       'Neighbourhood_group', 'Neighbourhood', 'Latitube', 'Longtitube',
       'Country', 'Country_code', 'Instant_bookable', 'Cancellation_policy',
       'Room_type', 'Construction_year', 'Price', 'Service_fee',
       'Minimum_nights', 'Number_of_reviews', 'Last_review',
       'Reviews_per_month', 'Review_rate_number',
       'Calculated_host_listings_count', 'Availability_365', 'House_rules'],
      dtype='object')

In [14]:
df

,Id,Name,Host_id,Host_identity_verified,Host_name,Neighbourhood_group,Neighbourhood,Latitube,Longtitube,Country,...,Price,Service_fee,Minimum_nights,Number_of_reviews,Last_review,Reviews_per_month,Review_rate_number,Calculated_host_listings_count,Availability_365,House_rules
0,6598f7d1fa398e3d8899aa10,False,80014485718,False,False,False,False,0,0,False,...,$966,False,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0
1,6598f7d1fa398e3d8899aa11,False,52335172823,False,False,False,False,0,0,False,...,$142,False,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0
2,6598f7d1fa398e3d8899aa12,False,78829239556,True,False,False,False,0,0,False,...,$620,False,0.0,0.0,True,1.0,0.0,0.0,0.0,0.0
3,6598f7d1fa398e3d8899aa13,True,85098326012,False,False,False,False,0,0,False,...,$368,False,0.0,0.0,False,0.0,0.0,0.0,0.0,1.0
4,6598f7d1fa398e3d8899aa14,False,92037596077,False,False,False,False,0,0,False,...,$204,False,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102594,6598f7d7fa398e3d889b3ad2,False,12312296767,False,False,False,False,0,0,False,...,$844,False,0.0,0.0,True,1.0,0.0,0.0,0.0,0.0
102595,6598f7d7fa398e3d889b3ad3,False,77864383453,False,False,False,False,0,0,False,...,$837,False,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0
102596,6598f7d7fa398e3d889b3ad4,False,69050334417,False,False,False,False,0,0,False,...,$988,False,0.0,0.0,True,1.0,0.0,0.0,0.0,1.0
102597,6598f7d7fa398e3d889b3ad5,False,11160591270,False,False,False,False,0,0,False,...,$546,False,0.0,0.0,False,0.0,0.0,0.0,0.0,1.0


# Save "Airbnb" Data Csv File

In [15]:
df.to_csv('Airbnb.csv',index=False)